
# AAI612: Deep Learning & its Applications


*Notebook 2.1: Learning the XOR Functions*

<a href="https://colab.research.google.com/github/harmanani/AAI612/blob/main/Week2/Notebook2.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
The MIT License (MIT)

Copyright (c) 2021 NVIDIA

Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""

### XOR Gate

This lab illustrates how the learning algorithm for multilevel feedforward networks works in practice using a simple XOR problem.  We use a the following three-neuron network:

<img src="https://github.com/harmanani/AAI612/blob/main/Week2/images/xor_neural_network.png?raw=1" style="width: 400px;">

The above network shows neurons N0, N1, and N2 with the bias inputs omitted.  We do not also show the weights in the figure. We use tanh as an activation function for N0 and N1 and the logistic sigmoid function as an activation function for the output neuron N2, and we use MSE as the loss function.

The XOR gates are the basis of computation and are often called in "half-adders", the foundation of being able to add numbers together. Here's the truth table for XOR:

### XOR (Exclusive Or) Gate

<table>

<tr>
<th colspan="3">XOR gate truth table</th>
</tr>

<tr>
<th colspan="2">Input</th>
<th>Output</th>
</tr>

<tr>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>

<tr>
<td>0</td>
<td>1</td>
<td>1</td>
</tr>

<tr>
<td>1</td>
<td>0</td>
<td>1</td>
</tr>

<tr>
<td>1</td>
<td>1</td>
<td>0</td>
</tr>

</table>

### Init Code for XOR Learning Example

In [9]:
import numpy as np

np.random.seed(3) # To make repeatable
LEARNING_RATE = 0.1
index_list = [0, 1, 2, 3] # Used to randomize order

# Define training examples.
x_train = [np.array([1.0, -1.0, -1.0]),
           np.array([1.0, -1.0, 1.0]),
           np.array([1.0, 1.0, -1.0]),
           np.array([1.0, 1.0, 1.0])]
y_train = [-1.0, -1.0, -1.0, 0.0] # Output (ground truth)

### Variables Needed to Track State of Neurons

In [10]:
def neuron_w(input_count):
    weights = np.zeros(input_count+1)
    for i in range(1, (input_count+1)):
        weights[i] = np.random.uniform(-1.0, 1.0)
    return weights

n_w = [neuron_w(2), neuron_w(2), neuron_w(2)]
n_y = [0, 0, 0]
n_error = [0, 0, 0]

### Helper Functions for Backpropagation:

In [11]:
def show_learning():
    print('Current weights:')
    for i, w in enumerate(n_w):
        print('neuron ', i, ': w0 =', '%5.2f' % w[0],
              ', w1 =', '%5.2f' % w[1], ', w2 =',
              '%5.2f' % w[2])
    print('----------------')

def forward_pass(x):
    global n_y
    n_y[0] = np.tanh(np.dot(n_w[0], x)) # Neuron 0
    n_y[1] = np.tanh(np.dot(n_w[1], x)) # Neuron 1
    n2_inputs = np.array([1.0, n_y[0], n_y[1]]) # 1.0 is bias
    z2 = np.dot(n_w[2], n2_inputs)
    n_y[2] = 1.0 / (1.0 + np.exp(-z2))

def backward_pass(y_truth):
    global n_error
    error_prime = -(y_truth - n_y[2]) # Derivative of loss-func
    derivative = n_y[2] * (1.0 - n_y[2]) # Logistic derivative
    n_error[2] = error_prime * derivative
    derivative = 1.0 - n_y[0]**2 # tanh derivative
    n_error[0] = n_w[2][1] * n_error[2] * derivative
    derivative = 1.0 - n_y[1]**2 # tanh derivative
    n_error[1] = n_w[2][2] * n_error[2] * derivative

def adjust_weights(x):
    global n_w
    n_w[0] -= (x * LEARNING_RATE * n_error[0])
    n_w[1] -= (x * LEARNING_RATE * n_error[1])
    n2_inputs = np.array([1.0, n_y[0], n_y[1]]) # 1.0 is bias
    n_w[2] -= (n2_inputs * LEARNING_RATE * n_error[2])

### Training Loop to Learn the XOR Function with Backpropagation

In [12]:
# Network training loop.
all_correct = False
while not all_correct: # Train until converged
    all_correct = True
    np.random.shuffle(index_list) # Randomize order
    for i in index_list: # Train on all examples
        forward_pass(x_train[i])
        backward_pass(y_train[i])
        adjust_weights(x_train[i])
        show_learning() # Show updated weights
    for i in range(len(x_train)): # Check if converged
        forward_pass(x_train[i])
        print('x1 =', '%4.1f' % x_train[i][1], ', x2 =',
              '%4.1f' % x_train[i][2], ', y =',
              '%.4f' % n_y[2])
        if(((y_train[i] < 0.5) and (n_y[2] >= 0.5))
                or ((y_train[i] >= 0.5) and (n_y[2] < 0.5))):
            all_correct = False

Current weights:
neuron  0 : w0 = -0.02 , w1 =  0.08 , w2 =  0.44
neuron  1 : w0 = -0.02 , w1 = -0.44 , w2 =  0.04
neuron  2 : w0 = -0.03 , w1 =  0.80 , w2 =  0.81
----------------
Current weights:
neuron  0 : w0 = -0.04 , w1 =  0.10 , w2 =  0.46
neuron  1 : w0 = -0.05 , w1 = -0.41 , w2 =  0.07
neuron  2 : w0 = -0.07 , w1 =  0.81 , w2 =  0.79
----------------
Current weights:
neuron  0 : w0 = -0.05 , w1 =  0.09 , w2 =  0.45
neuron  1 : w0 = -0.05 , w1 = -0.42 , w2 =  0.06
neuron  2 : w0 = -0.08 , w1 =  0.81 , w2 =  0.80
----------------
Current weights:
neuron  0 : w0 = -0.08 , w1 =  0.12 , w2 =  0.42
neuron  1 : w0 = -0.08 , w1 = -0.40 , w2 =  0.03
neuron  2 : w0 = -0.12 , w1 =  0.80 , w2 =  0.78
----------------
x1 = -1.0 , x2 = -1.0 , y = 0.4143
x1 = -1.0 , x2 =  1.0 , y = 0.5790
x1 =  1.0 , x2 = -1.0 , y = 0.3148
x1 =  1.0 , x2 =  1.0 , y = 0.4755
Current weights:
neuron  0 : w0 = -0.11 , w1 =  0.15 , w2 =  0.40
neuron  1 : w0 = -0.11 , w1 = -0.37 , w2 =  0.01
neuron  2 : w0 = -0.1